In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import inshallah as ins
from sklearn.impute import KNNImputer
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
import tensorflow as tf

ya allah


In [32]:
data = pd.read_csv("../CSV/big_files/ais_train.csv", sep="|")
vessels = pd.read_csv("../CSV/vessels.csv", sep="|")
test_og = pd.read_csv("../CSV/ais_test.csv")

vessels.head()

,shippingLineId,vesselId,CEU,DWT,GT,NT,vesselType,breadth,depth,draft,enginePower,freshWater,fuel,homePort,length,maxHeight,maxSpeed,maxWidth,rampCapacity,yearBuilt
0,61a8e672f9cba188601e84ab,61e9f38eb937134a3c4bfd8b,6500,21200.0,58684,17606.0,83.0,32.0,22.20,NaN,0.0,NaN,NaN,OSLO,199.00,5.0,18.6,15.2,150.0,2000
1,61ec94f1a8cafc0e93f0e92a,61e9f38eb937134a3c4bfd8d,4902,12325.0,46800,NaN,83.0,31.0,NaN,NaN,14220.0,NaN,NaN,MONROVIA,182.00,NaN,NaN,NaN,NaN,2006
2,61e213d5d612676a0f0fb755,61e9f38eb937134a3c4bfd8f,5000,13059.0,46800,NaN,83.0,31.0,NaN,NaN,14220.0,NaN,NaN,SAINT JOHN'S,182.00,NaN,NaN,NaN,NaN,2010
3,61be24574ea00ae59d0fe388,61e9f38eb937134a3c4bfd91,4200,12588.0,39362,NaN,83.0,28.0,NaN,NaN,11060.0,NaN,NaN,NaN,167.00,NaN,NaN,NaN,NaN,2011
4,61a8e673f9cba188601e84ae,61e9f390b937134a3c4bfd93,7450,21052.0,75528,24391.0,83.0,37.2,22.23,NaN,13140.0,491.47,3236.78,Panama,199.98,NaN,NaN,NaN,NaN,2018


In [3]:
def createIntervals(data0, FEATURES, n=5):
    data = data0.copy()
    data.sort_values("time", inplace=True)
    data["day_time"] = data["time"].apply(lambda t: pd.Timestamp(str(t).split(" ")[0]))

    data["month"] = data["time"].apply(lambda t: t.month)
    data["day"] = data["time"].apply(lambda t: t.day)
    data["hour"] = data["time"].apply(lambda t: t.hour)

    data["time"] = (data["time"] - pd.Timestamp("2024-01-01 00:00:00")).dt.total_seconds()

    intervals, sizes, final = [], [], []
    
    for i, grp in data.groupby(pd.Grouper(key="day_time", freq=f"{n}D")):
        intervals.append(grp[FEATURES].values)
        sizes.append(len(grp[FEATURES].values))

    maxSize = max(sizes)

    for i in intervals:
        residual = maxSize - len(i)
        padded = np.pad(i, ((0, residual), (0,0)), mode="constant", constant_values=0)
        final.append(padded)

    return np.array(final), sizes

def normalize_lat_lon(lat, lon):
    """
    Normalizes extreme latitude and longitude values, ensuring:
    - Latitude is constrained between [-90, 90] with appropriate longitude shifts.
    - Longitude is wrapped to be within [-180, 180].
    """
    
    # Handle extreme latitude values
    while lat > 90 or lat < -90:
        if lat > 90:
            lat = 180 - lat
            lon += 180
        elif lat < -90:
            lat = -180 - lat
            lon += 180
    
    # Normalize longitude using modulo to bring it within [-180, 180]
    lon = ((lon + 180) % 360) - 180
    
    return lat, lon

def reconstruct(sol: pd.DataFrame) -> pd.DataFrame:
    sol["vesselId"] = last_int["vesselId"]
    sol["time"] = last_int["time"]

    not_last_int = cleaned[["time", "vesselId", "latitude", "longitude"]].head(cleaned.shape[0] - y_sizes[-1])
    sol["latitude"] = np.nan

    mongo = []

    for i in sol["vesselId"].unique():
        current = sol[sol["vesselId"] == i]
        last_lat = not_last_int[not_last_int["vesselId"] == i]["latitude"].iloc[-1]
        last_lon = not_last_int[not_last_int["vesselId"] == i]["longitude"].iloc[-1]

        current["latitude"] = current["delta_lat"].cumsum() + last_lat
        current["longitude"] = current["delta_lon"].cumsum() + last_lon

        for _,row in current.iterrows():
            mongo.append(row.to_dict())

    truth = pd.DataFrame(mongo)
    return truth


In [28]:
#Imputation
vesselData = vessels.copy()
imputer = KNNImputer(n_neighbors=15)

non_numeric = ["vesselId", "shippingLineId", "homePort"]
non = vesselData[non_numeric]
vesselData.drop(columns=non_numeric, inplace=True)

cols, ind = vesselData.columns, vesselData.index

vesselData = pd.DataFrame(imputer.fit_transform(vesselData), columns=cols, index=ind)

for i in non_numeric:
    vesselData[i] = non[i]

#Cleaning   
data = pd.merge(data, vesselData, how="left")
cleaned = ins.cleanUp(data, n=25)

cleaned["volume"] = cleaned["length"]*cleaned["breadth"]*cleaned["depth"]

cleaned.head()

,time,cog,sog,rot,heading,navstat,etaRaw,latitude,longitude,vesselId,...,delta_lon_lag_1,delta_lat_lag_2,delta_lon_lag_2,delta_lat_lag_3,delta_lon_lag_3,delta_lat_lag_4,delta_lon_lag_4,delta_lat_lag_5,delta_lon_lag_5,volume
31706,2024-01-01 02:12:34,4.382522,0.0,5,4.380776,5,12-30 20:00,-23.95986,-46.29805,61e9f3c4b937134a3c4bfec3,...,0.00005,0.00000,-0.00002,0.00001,-0.00002,0.00001,0.00002,0.00000,-0.00002,143377.920000
32999,2024-01-01 02:12:37,4.515167,0.0,0,1.431170,5,12-31 20:00,59.82107,22.95338,61e9f464b937134a3c4c0263,...,-0.00001,0.00000,0.00003,0.00000,-0.00002,-0.00001,0.00001,-0.00002,0.00000,103550.876000
23915,2024-01-01 02:16:36,2.876303,0.1,0,4.293510,5,12-25 15:00,40.72223,29.48063,61e9f3c6b937134a3c4bfed5,...,0.00000,0.00002,0.00000,0.00000,0.00000,-0.00001,-0.00001,0.00001,0.00000,143377.920000
6673,2024-01-01 02:17:41,2.153736,0.0,128,3.071779,5,12-16 01:00,40.71466,29.46602,61e9f46bb937134a3c4c02b3,...,0.00001,0.00000,-0.00001,0.00000,-0.00001,0.00000,0.00002,0.00001,-0.00001,107397.333333
11544,2024-01-01 02:17:50,2.638938,0.0,0,3.630285,5,12-31 18:30,-19.25027,146.83509,61e9f3bfb937134a3c4bfe9f,...,-0.00002,0.00003,0.00003,0.00000,-0.00001,-0.00001,0.00001,0.00001,0.00000,151487.740040


In [35]:
#Normalize
NORMAL = ["delta_time", "speed_from_prev", "dist_from_prev", "delta_lat_lag_1",
          "delta_lon_lag_1", "NT", "volume", "delta_lat", "delta_lon"]

lat_std, lat_mean = cleaned["delta_lat"].std(), cleaned["delta_lat"].mean()
lon_std, lon_mean = cleaned["delta_lon"].std(), cleaned["delta_lon"].mean()
cleaned[NORMAL] = (cleaned[NORMAL] - cleaned[NORMAL].mean()) / cleaned[NORMAL].std()

cleaned[NORMAL].head()

,delta_time,speed_from_prev,dist_from_prev,delta_lat_lag_1,delta_lon_lag_1,NT,volume,delta_lat,delta_lon
31706,-0.057064,-0.790411,-0.069269,-0.000646,-0.000875,0.071632,1.101676,-0.001493,-0.000756
32999,-0.075451,-0.790701,-0.069277,-0.000639,-0.000889,-0.824390,-1.059568,-0.001493,-0.000752
23915,-0.071737,-0.790370,-0.069272,-0.000646,-0.000886,0.071632,1.101676,-0.001486,-0.000749
6673,-0.064370,-0.790453,-0.069272,-0.000639,-0.000884,-1.266542,-0.850837,-0.001500,-0.000754
11544,-0.060778,-0.790459,-0.069271,-0.000660,-0.000891,-0.979132,1.541762,-0.001500,-0.000749


In [23]:
#Gen data
TARGETS = ["delta_lat", "delta_lon"]
FEATURES = list(set(NORMAL) - set(TARGETS))

X, X_sizes = createIntervals(cleaned, FEATURES)
y, y_sizes = createIntervals(cleaned, TARGETS)

X_train, X_test = X[:-1], X[-1].reshape((1, X.shape[1], X.shape[2]))
y_train, y_test = y[:-1], y[-1]

#Parameters
sequence_length = X.shape[1]
num_features = X.shape[2]
num_targets = y.shape[2]

In [24]:
#LSTM
def lstm():
    lstm = tf.keras.Sequential([
        tf.keras.layers.LSTM(64, input_shape=(sequence_length, num_features), return_sequences=True),
        tf.keras.layers.LSTM(32, return_sequences=True),
        tf.keras.layers.Dense(num_targets)
    ])

    lstm.compile(optimizer="adam", loss="mse")

    # Check shapes for consistency
    print(f"X_train shape: {X_train.shape}")  # Expected shape: (num_samples, sequence_length, num_features)
    print(f"y_train shape: {y_train.shape}")  # Expected shape: (num_samples, sequence_length, num_targets)

    #Training model
    lstm.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

    #Predict
    y_pred_lstm = lstm.predict(X_test)

    #Loss
    y_lstm = y_pred_lstm.reshape(y_test.shape)
    print(f"LSTM MSE: {mean_squared_error(y_test, y_lstm)}")

    return y_lstm

#RNN
def rnn():
    rnn = tf.keras.Sequential([
        tf.keras.layers.SimpleRNN(64, activation="relu", input_shape=(sequence_length, num_features), return_sequences=True),
        tf.keras.layers.SimpleRNN(32, return_sequences=True),
        tf.keras.layers.Dense(num_targets)
    ])

    rnn.compile(optimizer="adam", loss="mse")

    # Check shapes for consistency
    print(f"X_train shape: {X_train.shape}")  # Expected shape: (num_samples, sequence_length, num_features)
    print(f"y_train shape: {y_train.shape}")  # Expected shape: (num_samples, sequence_length, num_targets)

    #Training model
    rnn.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

    #Predict
    y_rnn = rnn.predict(X_test).reshape(y_test.shape)
    print(f"RNN MSE: {mean_squared_error(y_test, y_rnn)}")

    return y_rnn

#CNN
def cnn():
    cnn = tf.keras.Sequential([
        tf.keras.layers.Conv1D(filters=64, kernel_size=2, activation='relu', padding="same", input_shape=(sequence_length, num_features)),
        tf.keras.layers.Conv1D(filters=32, kernel_size=2, activation='relu', padding="same"),
        tf.keras.layers.Conv1D(filters=16, kernel_size=2, activation='relu', padding="same"),
        tf.keras.layers.Conv1D(filters=2, kernel_size=1) 
    ])

    cnn.compile(optimizer="adam", loss="mse")

    # Check shapes for consistency
    print(f"X_train shape: {X_train.shape}")  # Expected shape: (num_samples, sequence_length, num_features)
    print(f"y_train shape: {y_train.shape}")  # Expected shape: (num_samples, sequence_length, num_targets)

    #Training model
    cnn.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

    #Predict
    y_cnn = cnn.predict(X_test).reshape(y_test.shape)
    print(f"CNN MSE: {mean_squared_error(y_test, y_cnn)}")

    return y_cnn

#LSTM with more layers
def lstm_plus():
    lstm_plus = tf.keras.Sequential([
        tf.keras.layers.LSTM(64, input_shape=(sequence_length, num_features), return_sequences=True),
        tf.keras.layers.LSTM(64, input_shape=(sequence_length, num_features), return_sequences=True),
        tf.keras.layers.LSTM(32, input_shape=(sequence_length, num_features), return_sequences=True),
        tf.keras.layers.Dense(50),
        tf.keras.layers.Dense(num_targets)
    ])

    lstm_plus.compile(optimizer="adam", loss="mse")

    # Check shapes for consistency
    print(f"X_train shape: {X_train.shape}")  # Expected shape: (num_samples, sequence_length, num_features)
    print(f"y_train shape: {y_train.shape}")  # Expected shape: (num_samples, sequence_length, num_targets)

    #Training model
    lstm_plus.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

    #Predict
    y_lstm_plus = lstm_plus.predict(X_test).reshape(y_test.shape)
    print(f"LSTM Plus MSE: {mean_squared_error(y_test, y_lstm_plus)}")

    return y_lstm_plus

#Bidirectional
def bi():
    bi = tf.keras.Sequential([
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True), input_shape=(sequence_length, num_features)),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32, return_sequences=True)),
        tf.keras.layers.Dense(num_targets)
    ])

    bi.compile(optimizer="adam", loss="mse")

    # Check shapes for consistency
    print(f"X_train shape: {X_train.shape}")  # Expected shape: (num_samples, sequence_length, num_features)
    print(f"y_train shape: {y_train.shape}")  # Expected shape: (num_samples, sequence_length, num_targets)

    #Training model
    bi.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

    #Predict
    y_bi = bi.predict(X_test).reshape(y_test.shape)
    print(f"Bidirectional MSE: {mean_squared_error(y_test, y_bi)}")

    return y_bi

#Hybrid
def hybrid():
    hybrid = tf.keras.Sequential([
        tf.keras.layers.Conv1D(64, kernel_size=3, activation='relu', padding='same', input_shape=(sequence_length, num_features)),
        tf.keras.layers.Conv1D(32, kernel_size=3, activation='relu', padding='same'),
        tf.keras.layers.LSTM(64, return_sequences=True),
        tf.keras.layers.LSTM(34, return_sequences=True),
        tf.keras.layers.Dense(64, activation="relu"),
        tf.keras.layers.Dense(num_targets)
    ])

    hybrid.compile(optimizer="adam", loss="mse")

    # Check shapes for consistency
    print(f"X_train shape: {X_train.shape}")  # Expected shape: (num_samples, sequence_length, num_features)
    print(f"y_train shape: {y_train.shape}")  # Expected shape: (num_samples, sequence_length, num_targets)

    #Training model
    hybrid.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

    #Predict
    y_hybrid = hybrid.predict(X_test).reshape(y_test.shape)
    print(f"Hybrid MSE: {mean_squared_error(y_test, y_hybrid)}")

    return y_hybrid


In [25]:
# x = 0.7
# y_ensemble = x*y_cnn + (1-x)*y_hybrid
# print(f"Ensemble ({x}*CNN and {(1-x)}*LSTM) MSE: {mean_squared_error(y_test, y_ensemble)}")

y_hybrid = hybrid()


X_train shape: (25, 2953, 8)
y_train shape: (25, 2953, 2)
Epoch 1/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 9s 9s/step - loss: 0.5371 - val_loss: 0.8888
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step - loss: 0.5336 - val_loss: 0.8888
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - loss: 0.5334 - val_loss: 0.8890
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step - loss: 0.5333 - val_loss: 0.8894
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - loss: 0.5332 - val_loss: 0.8897
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step - loss: 0.5331 - val_loss: 0.8898
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - loss: 0.5330 - val_loss: 0.8899
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - loss: 0.5329 - val_loss: 0.8901
Epoch 9/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - loss: 0.5328 - val_loss: 0.8904
Epoch 10/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step - loss: 0.5327 - val_loss: 0.8908
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 751ms/step
Hybrid MSE: 0.3562365994535128


In [26]:
last_int = cleaned[["time", "vesselId", "latitude", "longitude" ,"delta_lat", "delta_lon"]].tail(y_sizes[-1]).reset_index(drop=True)
prepred = pd.DataFrame(y_hybrid[0:y_sizes[-1]], columns=["delta_lat", "delta_lon"])
pretruth = pd.DataFrame(y_test[0:y_sizes[-1]], columns=["delta_lat", "delta_lon"])

#Un normalize
prepred["delta_lat"] = (prepred["delta_lat"] * lat_std) + lat_mean
prepred["delta_lon"] = (prepred["delta_lon"] * lon_std) + lon_mean

pretruth["delta_lat"] = (pretruth["delta_lat"] * lat_std) + lat_mean
pretruth["delta_lon"] = (pretruth["delta_lon"] * lon_std) + lon_mean

#Reconstruction
truth = reconstruct(pretruth)
pred = reconstruct(prepred)

print(truth["vesselId"].unique())


['61e9f41bb937134a3c4c0087' '61e9f468b937134a3c4c028f'
 '61e9f3aab937134a3c4bfe13' '61e9f3adb937134a3c4bfe39'
 '61e9f3b4b937134a3c4bfe73' '61e9f464b937134a3c4c0263'
 '61e9f46bb937134a3c4c02b3' '61e9f3e6b937134a3c4bff6d'
 '61e9f3f3b937134a3c4bffa9' '61e9f3c4b937134a3c4bfec3'
 '61e9f3bfb937134a3c4bfe9f' '61e9f45bb937134a3c4c0221'
 '61e9f3c9b937134a3c4bfef7' '61e9f3acb937134a3c4bfe23'
 '61e9f3aab937134a3c4bfe0b']


In [36]:
vessel = "61e9f41bb937134a3c4c0087"

fig = (
    ins.visualize_vessel_movements(truth[truth["vesselId"] == vessel])
)
fig.show()

fig = (
    ins.visualize_vessel_movements(cleaned[cleaned["vesselId"] == vessel])
)
fig.show()

fig = (
    ins.visualize_vessel_movements(pred[pred["vesselId"] == vessel])
)
fig.show()

In [43]:
test = test_og.copy()

test.drop(columns=["scaling_factor"], inplace=True)
test["time"] = (pd.to_datetime(test["time"]) - pd.Timestamp("2024-01-01 00:00:00")).dt.total_seconds()
test = pd.merge(test, vesselData, how="left")

test["volume"] = test["length"] * test["breadth"] * test["depth"]

for c in NORMAL:
    if c not in test.columns:
        if c!="delta_lat" and c!="delta_lon":
            test[c] = np.nan

test_list = []
for i in test["vesselId"].unique():
    boat = test[test["vesselId"] == i]
    
    for c in NORMAL:
        if c!="delta_lat" and c!="delta_lon":
            boat.at[0, c] = cleaned[cleaned["vesselId"] == i][c].iloc[-1]

    for _,row in boat.iterrows():
        test_list.append(row.to_dict())

test_df = pd.DataFrame(test_list, columns = test.columns)

test_df.head()

IndexError: single positional indexer is out-of-bounds

In [34]:
print(NORMAL)

['delta_time', 'speed_from_prev', 'dist_from_prev', 'delta_lat_lag_1', 'delta_lon_lag_1', 'dist_last_port', 'NT', 'volume', 'delta_lat', 'delta_lon']
